# Trend analysis: Check significant increasing of summer temperature with ERA5-Land monthly aggregated data

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pymannkendall as mk
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Set variables for request

In [ ]:
# Time
year_start = 1997
year_stop = 2022

## Set the data request

In [ ]:
collection_id = "reanalysis-era5-land-monthly-means"
request = {
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "time": "00:00",
    "area": [44, -10, 36, 0],
}

## Download data

In [ ]:
ds = download.download_and_transform(collection_id, request, chunks={"year": 1})
da = ds["t2m"]
with xr.set_options(keep_attrs=True):
    da -= 273.15
da.attrs["units"] = "°C "

## Plot time mean

In [ ]:
da_time_mean = diagnostics.time_weighted_mean(da)
_ = plot.projected_map(da_time_mean.where(da_time_mean), projection=ccrs.PlateCarree())

## Plot annual spatial mean

In [ ]:
da_spatial_mean = diagnostics.spatial_weighted_mean(da)
da_annual_mean = diagnostics.annual_weighted_mean(da_spatial_mean)
ax = da_annual_mean.to_pandas().plot.bar(x=da_spatial_mean["time"].dt.year)
ax.set_ylabel(f"{da_annual_mean.attrs['long_name']} [{da_annual_mean.attrs['units']}]")
ax.bar_label(ax.containers[0], rotation=90)

trend, h, p, z, tau, s, var_s, slope, intercept = mk.original_test(da_annual_mean)
is_significant = p < 0.05
print(f"The trend is{'' if is_significant else ' NOT'} significant.")
print(f"Trend: {slope}")